In [4]:
# >>> import library <<<
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import re

# >>> set chrome_driver options <<<
chrome_options = Options()
chrome_options.add_argument('--mute-audio') # disable audio
chrome_options.add_argument('--disable-features=Translate') # disable auto-translation of chrome
# chrome_options.add_argument("headless") # do not show window
chrome_options.add_experimental_option("detach", True) # keep browser open after script is completed

# >>> declare constants <<<
P = re.compile(r'SourID=([0-9A-Z]+)')

# >>> declare functions <<<
def replace_unusable_filename(file_name) : 
    l1 = ['\\', '/', ':', '*', '?', '"', '<', '>', '|']
    l2 = ['＼', '／', '∶', '＊', '？', '＂', '＜', '＞', '｜']
    
    for i in range(9) : 
        file_name = file_name.replace(l1[i], l2[i])
    
    return file_name

# create chrome instance with options
driver = webdriver.Chrome(options=chrome_options)

# open modern novel page
driver.get(url='http://www.davincimap.co.kr/davBase/Source/davSourList.jsp?job=C&tab=R&ClssID=DIRC000411&SelID=DIRC000420&worder=TITLE&work=TC')

# get writing's link
urls = []
for a in driver.find_elements(By.XPATH, '/html/body/div[2]/div/div[2]/table[6]/tbody/tr/td[3]/div/a') : 
    urls.append(a.get_attribute('href'))

# get writing's info and text
for i in range(0, len(urls)) : 
    # open writing page
    driver.get(url = urls[i])

    # get sourId
    sourId  = P.findall(urls[i])[0]

    # get writing's title
    title = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/table/tbody/tr[1]/td/h5').text
    title = title[2:len(title)-2]
    title = replace_unusable_filename(title)

    # get writing's author and number of pages
    info = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/table/tbody/tr[2]')
    author = '미상'
    try : 
        author = info.find_element(By.XPATH, './td[2]/div/div[2]').text
    except : 
        pass
    number_of_pages = 0
    isSinglePage = info.find_element(By.XPATH, './td[1]/span[3]').text
    if isSinglePage == '본문' : 
        number_of_pages = 1
    elif isSinglePage == '목차' : 
        tmp = info.find_element(By.XPATH, './td[1]/span[4]').text
        number_of_pages = int(tmp[5:len(tmp)-2])
    else : 
        print('can\'t find number of books -> ' + title)

    content = ''
    if number_of_pages == 1 : 
        for span in driver.find_elements(By.XPATH, '/html/body/table[1]/tbody/tr/td[1]/table[3]/tbody/tr[1]/td/table[last()]/tbody/tr/td/div/div[2]/span') : 
            content = content + span.text + ' ' 
    
    else : 
        for span in driver.find_elements(By.XPATH, '/html/body/table[1]/tbody/tr/td[1]/table[3]/tbody/tr[1]/td/table[last()]/tbody/tr/td/div/div[2]/span') : 
                content = content + span.text + ' ' 

        time.sleep(1)
        
        for page in range(2, number_of_pages + 1) : 
            driver.get(url = urls[i] + '&Lang=한글&Page=' + str(page))
            
            for span in driver.find_elements(By.XPATH, '/html/body/table[1]/tbody/tr/td[1]/table[3]/tbody/tr[1]/td/table[last()]/tbody/tr/td/div/div[2]/span') : 
                content = content + span.text + ' ' 

            time.sleep(1)

    file_path = 'C:\\jupyter_notebook\\Capstone Design\\data\\modern novel\\' + sourId + '&&&' + title + '.txt'
    with open(file_path, 'w', encoding='UTF-8') as file :
        file.write('sourid: ' + sourId + '\n')
        file.write('title: ' + title + '\n')
        file.write('author: ' + author + '\n')
        file.write('content: \n' + content + '\n')

    time.sleep(1)

can't find number of books -> 을지문덕 (乙支文德)


In [3]:
driver.get(url = 'http://www.davincimap.co.kr/davBase/Source/davSource.jsp?Job=Body&SourID=SOUR001477')

sourId  = 'SOUR001477'
title = '을지문덕 (乙支文德)'
author = '신채호'

content = ''
for span in driver.find_elements(By.XPATH, '/html/body/table[1]/tbody/tr/td[1]/table[3]/tbody/tr[1]/td/table[last()]/tbody/tr/td/div/div[2]/span') : 
    content = content + span.text + ' '

file_path = 'C:\\jupyter_notebook\\Capstone Design\\data\\modern novel\\' + sourId + '&&&' + title + '.txt'
with open(file_path, 'w', encoding='UTF-8') as file :
    file.write('sourid: ' + sourId + '\n')
    file.write('title: ' + title + '\n')
    file.write('author: ' + author + '\n')
    file.write('content: \n' + content + '\n')
